In [1]:
#import library
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\comp

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'E:\github\my upload\machine translation\hin.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()  #read all unique char of i/p
target_characters = set() #read all unique char of target
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n' #taking targert text only
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)  #len of i/p 
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])  #take max i/p length
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 2774
Number of unique input tokens: 70
Number of unique output tokens: 88
Max sequence length for inputs: 107
Max sequence length for outputs: 123


In [7]:
input_token_index = dict( #input_token_index have dict for assign token(number) to word
    [(char, i) for i, char in enumerate(input_characters)])  #assing tokens (number)to each char
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(  #assign 0 initally to all
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),  #encoder len, max seq,num of enc tokens
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), #decoder
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), #decoder
    dtype='float32')

In [8]:
# 1 hot rep of characters  (max length of i/p seq *number of unique token)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.



In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))  #i/p  shape define
encoder = LSTM(latent_dim, return_state=True)  #latent defining
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [9]:

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))  #i/p shape define
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Train on 2219 samples, validate on 555 samples
Epoch 1/100
2219/2219 [==============================] - 40s 18ms/step - loss: 1.2983 - accuracy: 0.7805 - val_loss: 1.7109 - val_accuracy: 0.6892
Epoch 2/100
2219/2219 [==============================] - 51s 23ms/step - loss: 0.9234 - accuracy: 0.8079 - val_loss: 1.3781 - val_accuracy: 0.6892
Epoch 3/100
2219/2219 [==============================] - 62s 28ms/step - loss: 0.8431 - accuracy: 0.8088 - val_loss: 1.3196 - val_accuracy: 0.6900
Epoch 4/100
2219/2219 [==============================] - 84s 38ms/step - loss: 0.8000 - accuracy: 0.8104 - val_loss: 1.2763 - val_accuracy: 0.6916
Epoch 5/100
2219/2219 [==============================] - 84s 38ms/step - loss: 0.7571 - accuracy: 0.8133 - val_loss: 1.2512 - val_accuracy: 0.6968
Epoch 6/100
2219/2219 [==============================] - 81s 37ms/step - loss: 0.7261 - accuracy: 0.8239 - val_loss: 1.1952 - val_accuracy: 0.7060
Epoch 7/100
2219/2219 [==============================] - 77s 35ms/step 

In [10]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Wow!
Decoded sentence: मैं तुम्हारे सितने का कर्या हैं।

-
Input sentence: Help!
Decoded sentence: मुझे अपने पैसे वापस चाहिए।

-
Input sentence: Jump.
Decoded sentence: मेरे पास आज साक से है क्या?

-
Input sentence: Jump.
Decoded sentence: मेरे पास आज साक से है क्या?

-
Input sentence: Jump.
Decoded sentence: मेरे पास आज साक से है क्या?

-
Input sentence: Hello!
Decoded sentence: मुझे अपने पैसे वापस चाहिए।

-
Input sentence: Hello!
Decoded sentence: मुझे अपने पैसे वापस चाहिए।

-
Input sentence: Cheers!
Decoded sentence: मेरे पास आज साक ले खते हो यया है।

-
Input sentence: Cheers!
Decoded sentence: मेरे पास आज साक ले खते हो यया है।

-
Input sentence: Got it?
Decoded sentence: मैं तुम्हें थोड़ी देर में फ़ोन करूँगा।

-
Input sentence: I'm OK.
Decoded sentence: मैं तुम्हारे सितने का कर्या हैं।

-
Input sentence: Awesome!
Decoded sentence: मैं तुम्हारे सितने का कर्या हैं।

-
Input sentence: Come in.
Decoded sentence: मेरे पास आज साक से है क्या?

-
Input sentence: Get out!
